#### Challenge 55:  MD4 Collisions

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
from Crypto.Random import random
from Crypto.Cipher import AES
from Crypto.Cipher import Blowfish
import hashlib
import math
import cryptopals as cp
import pdb

<div class="alert alert-block alert-info">   

MD4 is a `128-bit` cryptographic hash function, meaning it should take a work factor of roughly `2^64` to find collisions.

It turns out we can do much better.

The paper "Cryptanalysis of the Hash Functions MD4 and RIPEMD" by Wang et al details a cryptanalytic attack that lets us find collisions in `2^8` or less.

Given a message block `M`, Wang outlines a strategy for finding a sister message block `M'`, differing only in a few bits, that will collide with it. Just so long as a short set of conditions holds true for `M`.

What sort of conditions? Simple bitwise equalities within the intermediate hash function state, e.g. `a[1][6] = b[0][6]`. This should be read as: "the sixth bit (zero-indexed) of `a[1]` (i.e. the first update to `a`) should equal the sixth bit of `b[0]` (i.e. the initial value of `b`)".

It turns out that a lot of these conditions are trivial to enforce. To see why, take a look at the first (of three) rounds in the MD4 compression function. In this round, we iterate over each word in the message block sequentially and mix it into the state. So we can make sure all our first-round conditions hold by doing this:

```
# calculate the new value for a[1] in the normal fashion
a[1] = (a[0] + f(b[0], c[0], d[0]) + m[0]).lrot(3)

# correct the erroneous bit
a[1] ^= ((a[1][6] ^ b[0][6]) << 6)

# use algebra to correct the first message block
m[0] = a[1].rrot(3) - a[0] - f(b[0], c[0], d[0])
```
    
Simply ensuring all the first round conditions puts us well within the range to generate collisions, but we can do better by correcting some additional conditions in the second round. This is a bit trickier, as we need to take care not to stomp on any of the first-round conditions.

Once you've adequately massaged `M`, you can simply generate `M'` by flipping a few bits and test for a collision. A collision is not guaranteed as we didn't ensure every condition. But hopefully we got enough that we can find a suitable `(M, M')` pair without too much effort.

Implement Wang's attack.

</div>

---

I'll start by implementing MD4 in Python to get a better handle on what we're doing here.  Based on [RFC 1320](https://tools.ietf.org/html/rfc1320) by Rivest and the paper by Wang.

In [2]:
def bitget(x, n):
    """Return bit #n of x"""
    return (x >> n) & 1

def bit_in_place(x, n):
    """Return bit #n of x in its original bit position"""
    return (x & 2**n)

def bitset(x, n, bv):
    """Set bit #n of x to bv"""
    if bv==1:
        x |= 2**n
    else:
        x ^= bit_in_place(x, n)
    return(x)

def lrot_32(n, d):
    """Circular rotate left.  Python only natively supports non-circular shift."""
    return ( (n << d) | (n >> (32 - d)) )

def rrot_32(n, d):
    """Circular rotate right.  Python only natively supports non-circular shift."""
    return ( (n << (32 - d)) | (n >> d) )

def byte_swap(data, word_size):
    """ 
    Byte-swap's a byte string of words.  
    Specify word-length in bytes.
    """
    
    bs_data = [0]*len(data)
    for ii in range(0, len(data), word_size):
        bs_data[ii:ii+word_size] = data[ii:ii+4][::-1]
    return(bytes(bs_data))

In [3]:
# Define MD4 Auxilliary Functions:
MGK_1 = 0x5a827999
MGK_2 = 0x6ed9eba1

W = [ 
        [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15], 
        [0,  4,  8, 12,  1,  5,  9, 13,  2,  6, 10, 14,  3,  7, 11, 15],
        [0,  8,  4, 12,  2, 10,  6, 14,  1,  9,  5, 13,  3, 11,  7, 15] 
    ]   

S = [ 
        [3, 7, 11, 19],
        [3, 5,  9, 13],
        [3, 9, 11, 15]
    ]    

def F(X, Y, Z):         
    return (X & Y) | (~X & Z)

def G(X, Y, Z):
    return (X & Y) | (X & Z) | (Y & Z)

def H(X, Y, Z):
    return (X ^ Y ^ Z)

def phi(j, a, b, c, d, w, s):      
        
    if j == 0:            
        x = lrot_32(((a + F(b, c, d) + w) % 2**32), s)        
    elif j ==  1:            
        x = lrot_32(((a + G(b, c, d) + w + MGK_1) % 2**32), s)            
    elif j == 2:            
        x = lrot_32(((a + H(b, c, d) + w + MGK_2) % 2**32), s)                 
    return(x % 2**32)

def un_phi(j, a, b, c, d, T, s):
    
    if j==0:        
        w = (rrot_32(T, s) - a - F(b, c, d)) 
    elif j == 1:    
        w = (rrot_32(T, s) - a - G(b, c, d) - MGK_1) 
    elif j == 2:
        w = (rrot_32(T, s) - a - H(b, c, d) - MGK_2)
        
    return (w % 2**32)

def apply_corrections(var1, var2, corrections):
    
    #pdb.set_trace()
    for c_idx in range(32):
        if corrections[c_idx] != '.':
            if corrections[c_idx] == '0':
                var1 = bitset(var1, c_idx, 0)
            elif corrections[c_idx] == '1':
                var1 = bitset(var1, c_idx, 1)
            elif corrections[c_idx] == '=':
                var1 = bitset(var1, c_idx, bitget(var2, c_idx))
            else:
                raise(ValueError('Invalid Wang Rule'))
    return(var1)

In [4]:
def MD4_get_words(data, endianness='little'):
    """ Convert MD4 message data (512 bits / 64 bytes) to 32-bit words"""
    M = []
    for ii in range(0, len(data), 4):
        word = int.from_bytes(data[ii:ii+4], byteorder=endianness, signed=False)
        M.append(word)
        
    return(M)

In [5]:
def MD4_get_data(M, endianness='little'):
    """ Convert 16 x 32-bit MD4 words back to bytes"""
    data = b''
    for ii in range(16):
        M[ii] = M[ii] % 2**32
        data += M[ii].to_bytes(4, endianness)
        
    return(data)

In [6]:
def MD4_pad_data(data, endianness='little'):
    
    if isinstance(data, str):
        data = data.encode()
    
    # Step 1:  Append padding bits.  Single 1-bit + 0-bits so that
    #          length of message is congruent to 448 mod 512.
    #          I'll assume we're always passed a string of bytes.
    
    bit_bength = len(data)*8
    
    # append 1-bit = 0x80 
    data += b'\x80' # Hex 0x80 = 0b10000000
    
    data_ben = len(data) % 64
    padding_ben = (56 - data_ben) % 64
    
    data += b'\x00'*padding_ben
    
    # Step 2:  Append length.  64-bit representation before padding.
    
    data += bit_bength.to_bytes(8, endianness, signed=False)
    
    return(data)

In [7]:
def back_to_M(A,B,C,D):

    # Now back out the modified M
    M = [0]*16
    M[0] = (rrot_32(A[1], 3)  - A[0] - F(B[0], C[0], D[0])) % 2**32
    M[1] = (rrot_32(D[1], 7)  - D[0] - F(A[1], B[0], C[0])) % 2**32
    M[2] = (rrot_32(C[1], 11) - C[0] - F(D[1], A[1], B[0])) % 2**32
    M[3] = (rrot_32(B[1], 19) - B[0] - F(C[1], D[1], A[1])) % 2**32

    M[4] = (rrot_32(A[2], 3)  - A[1] - F(B[1], C[1], D[1])) % 2**32
    M[5] = (rrot_32(D[2], 7)  - D[1] - F(A[2], B[1], C[1])) % 2**32
    M[6] = (rrot_32(C[2], 11) - C[1] - F(D[2], A[2], B[1])) % 2**32
    M[7] = (rrot_32(B[2], 19) - B[1] - F(C[2], D[2], A[2])) % 2**32

    M[8] = (rrot_32(A[3], 3)   - A[2] - F(B[2], C[2], D[2])) % 2**32
    M[9] = (rrot_32(D[3], 7)   - D[2] - F(A[3], B[2], C[2])) % 2**32
    M[10] = (rrot_32(C[3], 11) - C[2] - F(D[3], A[3], B[2])) % 2**32
    M[11] = (rrot_32(B[3], 19) - B[2] - F(C[3], D[3], A[3])) % 2**32

    M[12] = (rrot_32(A[4], 3)  - A[3] - F(B[3], C[3], D[3])) % 2**32
    M[13] = (rrot_32(D[4], 7)  - D[3] - F(A[4], B[3], C[3])) % 2**32
    M[14] = (rrot_32(C[4], 11) - C[3] - F(D[4], A[4], B[3])) % 2**32
    M[15] = (rrot_32(B[4], 19) - B[3] - F(C[4], D[4], A[4])) % 2**32

    return(M)

In [8]:
def MD4_get_IVs(M):

    # Step 4:  Process Message in blocks of 16 32-bit words (512 bits ea)
    
    # Run the compression algorithm.  Loop for each block of 512 bits until
    # full message is consumed.
    
    #W = [ [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15], 
    #      [0,  4,  8, 12,  1,  5,  9, 13,  2,  6, 10, 14,  3,  7, 11, 15],
    #      [0,  8,  4, 12,  2, 10,  6, 14,  1,  9,  5, 13,  3, 11,  7, 15] ]   
    
    #S = [ [3, 7, 11, 19],
    #      [3, 5,  9, 13],
    #      [3, 9, 11, 15]]    

    A, B, C, D = [0x67452301], [0xefcdab89], [0x98badcfe], [0x10325476]
    
    N = len(M)//16

    for kk in range(N):
        
        AA, BB, CC, DD = A[-1], B[-1], C[-1], D[-1]
        X = M[16*kk:16*(kk+1)]
        
        for jj in range(3):        
            for ii in range(4):
                A.append(phi(jj, A[-1], B[-1], C[-1], D[-1], X[W[jj][4*ii+0]], S[jj][0]))
                D.append(phi(jj, D[-1], A[-1], B[-1], C[-1], X[W[jj][4*ii+1]], S[jj][1]))
                C.append(phi(jj, C[-1], D[-1], A[-1], B[-1], X[W[jj][4*ii+2]], S[jj][2]))
                B.append(phi(jj, B[-1], C[-1], D[-1], A[-1], X[W[jj][4*ii+3]], S[jj][3]))
     
        A.append((A[-1] + AA) % 2**32)
        B.append((B[-1] + BB) % 2**32)
        C.append((C[-1] + CC) % 2**32)
        D.append((D[-1] + DD) % 2**32)

    return(A, B, C, D)

In [9]:
def MD4(data, do_padding = True, endianness = 'little'):
    
    """ 
    Modified my MD4 implementation to better track with the notation used
    in Wang's paper -- and to keep intermediate results for all loop iterations
    within a round.  Could also modify to retain intermediates across rounds if
    needed.
    """
    if do_padding:
        M = MD4_get_words(MD4_pad_data(data), endianness)
    else:
        if len(data) < 64:
            data += b'\x00'*(64-len(data))
        M = MD4_get_words(data, endianness)
        
    A, B, C, D = MD4_get_IVs(M)           
    #pdb.set_trace()
    digest = A[-1].to_bytes(4, endianness) + \
             B[-1].to_bytes(4, endianness) + \
             C[-1].to_bytes(4, endianness) + \
             D[-1].to_bytes(4, endianness)
   
    return(digest)

In [10]:
# Check against the RFC 1320 test suite:

assert(MD4('').hex() == '31d6cfe0d16ae931b73c59d7e0c089c0')
assert(MD4('a').hex() == 'bde52cb31de33e46245e05fbdbd6fb24')
assert(MD4('abc').hex() == 'a448017aaf21d8525fc10ae87aa6729d')
assert(MD4('message digest').hex() == 'd9130a8164549fe818874806e1c7014b')
assert(MD4('abcdefghijklmnopqrstuvwxyz').hex() == 'd79e1c308aa5bbcdeea8ed63df412da9')
assert(MD4('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789').hex() == '043f8582f241db351ce627e153e7f0e4')
assert(MD4('12345678901234567890123456789012345678901234567890123456789012345678901234567890').hex() == \
      'e33b4ddc9c38f2199c3e7b164fcc0536')

print('If you can see this, all the tests passed.')

If you can see this, all the tests passed.


---

From the Wang paper, the attack includes 3 parts:
    
1. Find a collision differential in which `M` and `M'` produces a collision
2. Derive a set of sufficient conditions which ensure the collision differential to hold.
3. For any random message `M`, make some modification to `M` such that almost all the sufficient conditions hold.

---

In [11]:
# The examples in Table 3 of Wang's paper use big-endian byte order for the examples that
# include message padding.  My MD4 uses little-endian.  I need a byte-swap function to fix 
# the byte ordering...

M_1_be = bytes.fromhex(
    '4d7a9c83 56cb927a b9d5a578 57a7a5ee de748a3c dcc366b3 b683a020 3b2a5d9f' 
    'c69d71b3 f9e99198 d79f805e a63bb2e8 45dd8e31 97e31fe5 2794bf08 b9e8c3e9')
M_1_c_be = bytes.fromhex(
    '4d7a9c83 d6cb927a 29d5a578 57a7a5ee de748a3c dcc366b3 b683a020 3b2a5d9f' 
    'c69d71b3 f9e99198 d79f805e a63bb2e8 45dc8e31 97e31fe5 2794bf08 b9e8c3e9')
M_1 = byte_swap(M_1_be, 4)
M_1_c = byte_swap(M_1_c_be, 4)

H_1 = bytes.fromhex('4d7e6a1d efa93d2d de05b45d 864c429b')
H_1_np = bytes.fromhex('5f5c1a0d 71b36046 1b5435da 9b0d807a')

M_2_be = bytes.fromhex(
    '4d7a9c83 56cb927a b9d5a578 57a7a5ee de748a3c dcc366b3 b683a020 3b2a5d9f' 
    'c69d71b3 f9e99198 d79f805e a63bb2e8 45dd8e31 97e31fe5 f713c240 a7b8cf69')
M_2_c_be = bytes.fromhex(
    '4d7a9c83 d6cb927a 29d5a578 57a7a5ee de748a3c dcc366b3 b683a020 3b2a5d9f' 
    'c69d71b3 f9e99198 d79f805e a63bb2e8 45dc8e31 97e31fe5 f713c240 a7b8cf69')

M_2 = byte_swap(M_2_be, 4)
M_2_c = byte_swap(M_2_c_be, 4)

H_2 = bytes.fromhex('c6f3b3fe 1f4833e0 697340fb 214fb9ea')
H_2_np = bytes.fromhex('e0f76122 c429c56c ebb5e256 0b809793')

# Check paper examples using default settings (padding + little endian byte-order)
assert(MD4(M_1) == H_1)
assert(MD4(M_1_c) == H_1)
assert(MD4(M_2) == H_2)
assert(MD4(M_2_c) == H_2)

# Check paper examples using alternate settings (no padding + big endian byte-order)
assert(MD4(M_1_be, False, 'big') == H_1_np)
assert(MD4(M_1_c_be, False, 'big') == H_1_np)
assert(MD4(M_2_be, False, 'big') == H_2_np)
assert(MD4(M_2_c_be, False, 'big') == H_2_np)

print('Table 3 Tests passed')

Table 3 Tests passed


---

We've verified that the MD4 implementation works with the sample data...now let's see if we can follow how the colliding messages were found from the originals.

I'll try to follow the notation used in Wang's paper

---

In [12]:
def Wang_Msg_Differential(data, endianness='little'):
    
    M = MD4_get_words(data, endianness)
    M[1] = (M[1] + 2**31) % 2**32
    M[2] = (M[2] + (2**31 - 2**28)) % 2**32
    M[12] = (M[12] - 2**16) % 2**32
    
    return(MD4_get_data(M, endianness))

In [13]:
def check_conditions(A,B,C,D):
    
    assert(bitget(A[1], 6) == bitget(B[0], 6))
    assert(bitget(D[1], 6) == 0)
    assert(bitget(D[1], 7) == bitget(A[1], 7))
    assert(bitget(D[1], 10) == bitget(A[1], 10))
    assert(bitget(C[1], 6) == 1)
    assert(bitget(C[1], 7) == 1)
    assert(bitget(C[1], 10) == 0)
    assert(bitget(C[1], 25) == bitget(D[1], 25))
    assert(bitget(B[1], 6) == 1)
    assert(bitget(B[1], 7) == 0)
    assert(bitget(B[1], 10) == 0)
    assert(bitget(B[1], 25) == 0)
    assert(bitget(A[2], 7) == 1)
    assert(bitget(A[2], 10) == 1)
    assert(bitget(A[2], 13) == bitget(B[1], 13))
    assert(bitget(A[2], 25) == 0)
    assert(bitget(D[2], 13) == 0)
    assert(bitget(D[2], 18) == bitget(A[2], 18))
    assert(bitget(D[2], 19) == bitget(A[2], 19))
    assert(bitget(D[2], 20) == bitget(A[2], 20))
    assert(bitget(D[2], 21) == bitget(A[2], 21))
    assert(bitget(D[2], 25) == 1)
    assert(bitget(C[2], 12) == bitget(D[2], 12))
    assert(bitget(C[2], 13) == 0)
    assert(bitget(C[2], 14) == bitget(D[2], 14))
    assert(bitget(C[2], 18) == 0)
    assert(bitget(C[2], 19) == 0)
    assert(bitget(C[2], 20) == 1)
    assert(bitget(C[2], 21) == 0)
    assert(bitget(B[2], 12) == 1)
    assert(bitget(B[2], 13) == 1)
    assert(bitget(B[2], 14) == 0)
    assert(bitget(B[2], 16) == bitget(C[2], 16))
    assert(bitget(B[2], 18) == 0)
    assert(bitget(B[2], 19) == 0)
    assert(bitget(B[2], 20) == 0)
    assert(bitget(B[2], 21) == 0)
    assert(bitget(A[3], 12) == 1)
    assert(bitget(A[3], 13) == 1)
    assert(bitget(A[3], 14) == 1)
    assert(bitget(A[3], 16) == 0)
    assert(bitget(A[3], 18) == 0)
    assert(bitget(A[3], 19) == 0)
    assert(bitget(A[3], 20) == 0)
    assert(bitget(A[3], 21) == 1)
    assert(bitget(A[3], 22) == bitget(B[2], 22))
    assert(bitget(A[3], 25) == bitget(B[2], 25))
    assert(bitget(D[3], 12) == 1)
    assert(bitget(D[3], 13) == 1)
    assert(bitget(D[3], 14) == 1)
    assert(bitget(D[3], 16) == 0)
    assert(bitget(D[3], 19) == 0)
    assert(bitget(D[3], 20) == 1)
    assert(bitget(D[3], 21) == 1)
    assert(bitget(D[3], 22) == 0)
    assert(bitget(D[3], 25) == 1)
    assert(bitget(D[3], 29) == bitget(A[3], 29))
    assert(bitget(C[3], 16) == 1)
    assert(bitget(C[3], 19) == 0)
    assert(bitget(C[3], 20) == 0)
    assert(bitget(C[3], 21) == 0)
    assert(bitget(C[3], 22) == 0)
    assert(bitget(C[3], 25) == 0)
    assert(bitget(C[3], 29) == 1)
    assert(bitget(C[3], 31) == bitget(D[3], 31))
    assert(bitget(B[3], 19) == 0)
    assert(bitget(B[3], 20) == 1)
    assert(bitget(B[3], 21) == 1)
    assert(bitget(B[3], 22) == bitget(C[3], 22))
    assert(bitget(B[3], 25) == 1)
    assert(bitget(B[3], 29) == 0)
    assert(bitget(B[3], 31) == 0)
    assert(bitget(A[4], 22) == 0)
    assert(bitget(A[4], 25) == 0)
    assert(bitget(A[4], 26) == bitget(B[3], 26))
    assert(bitget(A[4], 28) == bitget(B[3], 28))
    assert(bitget(A[4], 29) == 1)
    assert(bitget(A[4], 31) == 0)
    assert(bitget(D[4], 22) == 0)
    assert(bitget(D[4], 25) == 0)
    assert(bitget(D[4], 26) == 1)
    assert(bitget(D[4], 28) == 1)
    assert(bitget(D[4], 29) == 0)
    assert(bitget(D[4], 31) == 1)
    assert(bitget(C[4], 18) == bitget(D[4], 18))
    assert(bitget(C[4], 22) == 1)
    assert(bitget(C[4], 25) == 1)
    assert(bitget(C[4], 26) == 0)
    assert(bitget(C[4], 28) == 0)
    assert(bitget(C[4], 29) == 0)
    assert(bitget(B[4], 18) == 0)
    assert(bitget(B[4], 25) == 1)
    assert(bitget(B[4], 26) == 1)
    assert(bitget(B[4], 28) == 1)
    assert(bitget(B[4], 29) == 0)
    assert(bitget(B[4], 31) == bitget(C[4], 31))

In [14]:
# Setup matrix containing Wang's corrections for single-step modification (Table 6).
# These are the "constraints" we use to modify a random message so that the differential
# message is more likely to result in a hash collision
#
# Each row is a step in the MD4 algorithm.
# Each column is a bit of the value being computed during that 
# step, ordered from LSB (0) to MSB (31) (left to right).
#
# Note:  Wang's rules have been converted from Base 1 to Base 0.

#                           Bit #'ing
#              00000000001111111111222222222233
#              01234567890123456789012345678901 
Wang_Rules = ['......=.........................',
             '......0=..=.....................',
             '......11..0..............=......',
             '......10..0..............0......',
             '.......1..1..=...........0......',
             '.............0....====...1......',
             '............=0=...0010..........',
             '............110.=.0000..........',
             '............111.0.0001=..=......',
             '............111.0..0110..1...=..',
             '................1..0000..0...1.=',
             '...................011=..1...0.0',
             '......................0..0=.=1.0',
             '......................0..01.10.1',
             '..................=...1..10.00..',
             '..................0......11.10.=']

In [15]:
def Wang_Corrections(data, fixA5=True, fixD5=True, fixC5=True, fixB5=True, do_padding=False, endianness='little'):
    
    
    if do_padding:        
        M = MD4_get_words(MD4_pad_data(data), endianness)        
    else:        
        if len(data) < 64:
            data += b'\x00'*(64-len(data))            
        M = MD4_get_words(data, endianness) 
    
    A, B, C, D = MD4_get_IVs(M)
    
    jj = 0 # Constant...we're just doing the 1st round corrections.
    for ii in range(4):
        
        corrections = Wang_Rules[4*ii]
        A[ii+1] = phi(jj, A[ii], B[ii], C[ii], D[ii], M[W[jj][4*ii+0]], S[jj][0])
        A[ii+1] = apply_corrections(A[ii+1], B[ii], corrections)
        M[W[jj][4*ii+0]] = un_phi(jj, A[ii], B[ii], C[ii], D[ii], A[ii+1], S[jj][0])
        
        corrections = Wang_Rules[4*ii+1]
        D[ii+1] = phi(jj, D[ii], A[ii+1], B[ii], C[ii], M[W[jj][4*ii+1]], S[jj][1])
        D[ii+1] = apply_corrections(D[ii+1], A[ii+1], corrections)
        M[W[jj][4*ii+1]] = un_phi(jj, D[ii], A[ii+1], B[ii], C[ii], D[ii+1], S[jj][1])
        
        corrections = Wang_Rules[4*ii+2]      
        C[ii+1] = phi(jj, C[ii], D[ii+1], A[ii+1], B[ii], M[W[jj][4*ii+2]], S[jj][2])
        C[ii+1] = apply_corrections(C[ii+1], D[ii+1], corrections)
        M[W[jj][4*ii+2]] = un_phi(jj, C[ii], D[ii+1], A[ii+1], B[ii], C[ii+1], S[jj][2])
        
        corrections = Wang_Rules[4*ii+3]
        B[ii+1] = phi(jj, B[ii], C[ii+1],   D[ii+1],   A[ii+1],   M[W[jj][4*ii+3]], S[jj][3])
        B[ii+1] = apply_corrections(B[ii+1], C[ii+1], corrections)
        M[W[jj][4*ii+3]] = un_phi(jj, B[ii], C[ii+1],   D[ii+1],   A[ii+1], B[ii+1], S[jj][3])
    
    # Now the multi-message corrections.  I wasn't able to do these exactly the way Wang's paper
    # says -- the notation is quite confusing.  Instead, I followed this procedure:
    # 
    # 1.  Modify the intermediate value to meet all constraints in Table 6.
    # 2.  Back out the message byte that was directly involved in calculation of that intermediate
    #     (just use algebra to see what M[x] must be to produce that intermediate result, assuming
    #      all of the other intermediates are the same)
    # 3.  Find the first-round value that also directly depends on the modified message byte.  E.g.,
    #     for A[5], we modify M[0].  M[0] is intitially used to compute A[1], so we re-calculate A[1].
    # 4.  Find all of the calculations that directly use A[x] (A[1] in the example above), and 
    #     
    
    if fixA5:
        
        A, B, C, D = MD4_get_IVs(M)  

        A[5] = bitset(A[5], 18, bitget(C[4], 18))
        A[5] = bitset(A[5], 25, 1)
        A[5] = bitset(A[5], 26, 0)
        A[5] = bitset(A[5], 28, 1)
        A[5] = bitset(A[5], 31, 1)

        M[0] = rrot_32(A[5], 3) - A[4] - G(B[4], C[4], D[4]) - MGK_1

        A[1] = lrot_32( (A[0] + F(B[0], C[0], D[0]) + M[0]) % 2**32, 3)
        M[1] = rrot_32(D[1],  7) - D[0] - F(A[1], B[0], C[0]) % 2**32
        M[2] = rrot_32(C[1], 11) - C[0] - F(D[1], A[1], B[0]) % 2**32
        M[3] = rrot_32(B[1], 19) - B[0] - F(C[1], D[1], A[1]) % 2**32
        M[4] = rrot_32(A[2],  3) - A[1] - F(B[1], C[1], D[1]) % 2**32

    if fixD5:
        
        A, B, C, D = MD4_get_IVs(M)
        
        D[5] = bitset(D[5], 18, bitget(A[5], 18))
        D[5] = bitset(D[5], 25, bitget(B[4], 25))
        D[5] = bitset(D[5], 26, bitget(B[4], 26))
        D[5] = bitset(D[5], 28, bitget(B[4], 28))
        D[5] = bitset(D[5], 31, bitget(B[4], 31)) % 2**32

        M[4] = (rrot_32(D[5], 5) - D[4] - G(A[5], B[4], C[4]) - MGK_1) % 2**32
        A[2] = lrot_32( (A[1] + F(B[1], C[1], D[1]) + M[4]) % 2**32, 3) % 2**32

        M[5] = rrot_32(D[2],  7) - D[1] - F(A[2], B[1], C[1]) % 2**32
        M[6] = rrot_32(C[2], 11) - C[1] - F(D[2], A[2], B[1]) % 2**32
        M[7] = rrot_32(B[2], 19) - B[1] - F(C[2], D[2], A[2]) % 2**32
        M[8] = rrot_32(A[3],  3) - A[2] - F(B[2], C[2], D[2]) % 2**32
    
    if fixC5:
        
        A, B, C, D = MD4_get_IVs(M)

        C[5] = bitset(C[5], 25, bitget(D[5], 25))
        C[5] = bitset(C[5], 26, bitget(D[5], 26))
        C[5] = bitset(C[5], 28, bitget(D[5], 28))
        C[5] = bitset(C[5], 29, bitget(D[5], 29))
        C[5] = bitset(C[5], 31, bitget(D[5], 31)) % 2**32

        M[8] = (rrot_32(C[5], 9) - C[4] - G(D[5], A[5], B[4]) - MGK_1) % 2**32
        A[3] = lrot_32( (A[2] + F(B[2], C[2], D[2]) + M[8]) % 2**32, 3)
        
        M[9]  = rrot_32(D[3],  7)  - D[2] - F(A[3], B[2], C[2]) % 2**32
        M[10] = rrot_32(C[3], 11) - C[2] - F(D[3], A[3], B[2]) % 2**32
        M[11] = rrot_32(B[3], 19) - B[2] - F(C[3], D[3], A[3]) % 2**32
        M[12] = rrot_32(A[4],  3) - A[3] - F(B[3], C[3], D[3]) % 2**32
        
    if fixB5:
        
        A, B, C, D = MD4_get_IVs(M)
        B[5] = bitset(B[5], 28, bitget(C[5], 28))
        B[5] = bitset(B[5], 29, 1)
        B[5] = bitset(B[5], 31, 0)

        M[12] = (rrot_32(B[5], 13) - B[4] - G(C[5], D[5], A[5]) - MGK_1) % 2**32
        A[4] = lrot_32( (A[3] + F(B[3], C[3], D[3]) + M[12]) % 2**32, 3)
        
        M[13] = rrot_32(D[4], 7) - D[3] - F(A[4], B[3], C[3]) % 2**32
        M[14] = rrot_32(C[4], 11) - C[3] - F(D[4], A[4], B[3]) % 2**32
        M[15] = rrot_32(B[4], 19) - B[3] - F(C[4], D[4], A[4]) % 2**32   
        M[0] = rrot_32(A[5], 3) - A[4] - G(B[4], C[4], D[4]) - MGK_1 % 2**32
        
        # Change cascades..
        A[1] = lrot_32( (A[0] + F(B[0], C[0], D[0]) + M[0]) % 2**32, 3)
        M[1] = rrot_32(D[1],  7) - D[0] - F(A[1], B[0], C[0]) % 2**32
        M[2] = rrot_32(C[1], 11) - C[0] - F(D[1], A[1], B[0]) % 2**32
        M[3] = rrot_32(B[1], 19) - B[0] - F(C[1], D[1], A[1]) % 2**32
        M[4] = rrot_32(A[2],  3) - A[1] - F(B[1], C[1], D[1]) % 2**32  
    
    new_data = MD4_get_data(M, endianness)
    
    return(new_data)

In [16]:
def Wang_fixA5(data, do_padding=True, endianness='little'):
    
    """ 
    Implement Table 1 changes modify M for A[5] corrections.
    """
    if do_padding:
        M = MD4_get_words(MD4_pad_data(data, endianness))
    else:
        if len(data) < 64:
            data += b'\x00'*(64-len(data))
        M = MD4_get_words(data, endianness) 
        
    A, B, C, D = MD4_get_IVs(M)  
        
    A[5] = bitset(A[5], 18, bitget(C[4], 18))
    A[5] = bitset(A[5], 25, 1)
    A[5] = bitset(A[5], 26, 0)
    A[5] = bitset(A[5], 28, 1)
    A[5] = bitset(A[5], 31, 1)
    
    # 
    M[0] = rrot_32(A[5], 3) - A[4] - G(B[4], C[4], D[4]) - MGK_1
    
    A[1] = lrot_32( (A[0] + F(B[0], C[0], D[0]) + M[0]) % 2**32, 3)
    
    # Fix all the messages that preceed A[1]...
    M[1] = rrot_32(D[1],  7) - D[0] - F(A[1], B[0], C[0]) % 2**32
    M[2] = rrot_32(C[1], 11) - C[0] - F(D[1], A[1], B[0]) % 2**32
    M[3] = rrot_32(B[1], 19) - B[0] - F(C[1], D[1], A[1]) % 2**32
    M[4] = rrot_32(A[2],  3) - A[1] - F(B[1], C[1], D[1]) % 2**32
        
    # Check constraints...a5;19 = c4;19, a5;26 = 1, a5;27 = 0, a5;29 = 1, a5;32 = 1
    A,B,C,D = MD4_get_IVs(M)
    
    assert(bitget(A[5], 18) == bitget(C[4], 18))
    assert(bitget(A[5], 25) == 1)
    assert(bitget(A[5], 26) == 0)
    assert(bitget(A[5], 28) == 1)    
    assert(bitget(A[5], 31) == 1)
    
    return(MD4_get_data(M, endianness))

In [17]:
def Wang_fixD5(data, do_padding=True, endianness='little'):
    
    #d5;19 = a5;19,   d5;26 = b4;26,   d5;27 = b4;27,   d5;29 = b4;29,   d5;32 = b4;32
    
    if do_padding:
        M = MD4_get_words(MD4_pad_data(data, endianness))
    else:
        if len(data) < 64:
            data += b'\x00'*(64-len(data))
        M = MD4_get_words(data, endianness) 
        
    A, B, C, D = MD4_get_IVs(M)
    #print(A[0:6],B[0:6],C[0:6],D[0:6],'\n')
    
    D[5] = bitset(D[5], 18, bitget(A[5], 18))
    D[5] = bitset(D[5], 25, bitget(B[4], 25))
    D[5] = bitset(D[5], 26, bitget(B[4], 26))
    D[5] = bitset(D[5], 28, bitget(B[4], 28))
    D[5] = bitset(D[5], 31, bitget(B[4], 31))
    D[5] = D[5] % 2**32
    
    #     D[5] = lrot_32( (D[4] + G(A[5], B[4], C[4]) + M[4] + MGK_1) % 2**32, 5)
    M[4] = (rrot_32(D[5], 5) - D[4] - G(A[5], B[4], C[4]) - MGK_1) % 2**32
    A[2] = lrot_32( (A[1] + F(B[1], C[1], D[1]) + M[4]) % 2**32, 3) % 2**32
    
    M[5] = rrot_32(D[2],  7) - D[1] - F(A[2], B[1], C[1]) % 2**32
    M[6] = rrot_32(C[2], 11) - C[1] - F(D[2], A[2], B[1]) % 2**32
    M[7] = rrot_32(B[2], 19) - B[1] - F(C[2], D[2], A[2]) % 2**32
    M[8] = rrot_32(A[3],  3) - A[2] - F(B[2], C[2], D[2]) % 2**32
    
    A,B,C,D = MD4_get_IVs(M)
    #print(A[0:6],B[0:6],C[0:6],D[0:6])
    
    assert(bitget(D[5], 18) == bitget(A[5], 18))
    assert(bitget(D[5], 25) == bitget(B[4], 25))
    assert(bitget(D[5], 26) == bitget(B[4], 26))
    assert(bitget(D[5], 28) == bitget(B[4], 28))
    assert(bitget(D[5], 31) == bitget(B[4], 31))

    return(MD4_get_data(M, endianness))

---

The above code implements Wang's message modifications.  Now, let's try them using a bunch of random messages.  

---

---
### __It's working...__

Here are a couple of collisions found with the early code (before all of the multi-message modifications were working).

Original Message = d2423055e29921d5369ac5d22707fd98211649aa2433ddf96f9efeb510217e621fbf4d1bada3368a7b1408d86ba1a162a7c74f0cddf4d5fe7937d9621ac16e56
M = d242f064ea99214f359ac55426070515a016492c24ebdcf96f9ffeb50b217e841fa1591b4da2fa897b0bf8d762a1a162a7c7cf00dd74d1fff97fd7621acf6e56
M = d242f064ea9921cf359ac5c426070515a016492c24ebdcf96f9ffeb50b217e841fa1591b4da2fa897b0bf8d762a1a162a7c7ce00dd74d1fff97fd7621acf6e56
Hash = 157cbb9fbc52b66f83fee8904364751c
Number of Attempts:  208754

Original Message = 1e5b3a8ea3aa6b20c59668513fd10f71d8f586deb8902834d4752c83afc6145c508b765b2781cc1ae7eeaf0c6c12258454df3c135f94d0581de67be7ecc105da
M = 165b7a90a3aa6b6ec5966817ffd07f00e8f986ccb8702834d8722c83acc6147e508d6e5b27e1d01ae7eaaf0cac02258454df74045f94cc599d657be72cc2055a
M = 165b7a90a3aa6beec5966887ffd07f00e8f986ccb8702834d8722c83acc6147e508d6e5b27e1d01ae7eaaf0cac02258454df73045f94cc599d657be72cc2055a
Hash = 76a9e05832608f1a8cd1be6cc94b8165
Number of Attempts:  2155812


### With multi-message modifications:


After 74 collisions with corrections for everything except for B5, average # of random messages to find a new collision was 83723.  (~2**16th)

AFter 99 collisions with B5, average # of messages to find a collision was 91651.  Apparently doing something wrong on my B5 correction -- probably stomping on the corrections already done for the others.

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)